In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import numpy as np
import pandas as pd 

# LOAD DATA SET

In [6]:
dataEkster = pd.read_csv("/kaggle/input/datateksbaru/datateksbersih.csv")

In [10]:
dataEkster.isna().sum()

teks_bersih    27
dtype: int64

In [8]:
print(dataEkster[dataEkster['teks_bersih'].isna()])


      teks_bersih
9781          NaN
9792          NaN
10220         NaN
10318         NaN
10377         NaN
10398         NaN
11925         NaN
12048         NaN
13400         NaN
13592         NaN
13624         NaN
13679         NaN
13680         NaN
13876         NaN
13877         NaN
13878         NaN
13879         NaN
13942         NaN
13944         NaN
14018         NaN
14119         NaN
14123         NaN
14124         NaN
14125         NaN
14127         NaN
14135         NaN
14137         NaN


In [11]:
data = dataEkster.copy()

In [12]:
data = data.dropna(subset=['teks_bersih'])


# VECTORIZE

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import OneClassSVM

In [14]:
# Vectorizer
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(data['teks_bersih'])


ocsvm = OneClassSVM(kernel="rbf", nu=0.1, gamma="scale")
ocsvm.fit(X_train)

OneClassSVM(nu=0.1)

In [22]:
import joblib
joblib.dump(vectorizer, "vectorizer.pkl")

['vectorizer.pkl']

# UJI COBA MODEL

In [15]:
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
def deteksi_beritas(input_text):
    # Transform input
    X_input = vectorizer.transform([input_text])
    pred = ocsvm.predict(X_input)  # 1 = Fakta, -1 = Hoax

    # Hitung similarity dengan semua berita fakta
    similarities = cosine_similarity(X_input, X_train)
    idx_terdekat = similarities.argmax()  # ambil yang paling mirip
    berita_asli = data['teks_bersih'].iloc[idx_terdekat]
    skor_kemiripan = similarities[0, idx_terdekat] * 100  # jadi persen

    if pred[0] == 1:
        return "Fakta", berita_asli, skor_kemiripan
    else:
        return "Diduga Hoax", berita_asli, skor_kemiripan

In [17]:
hoax = "Anies Baswedan menghadiri acara milad BKMT di Istora Senayan, namun dalam sambutannya ia menyebut bahwa kegiatan pengajian justru membuat ibu-ibu semakin mudah terprovokasi oleh isu politik. Ia bahkan mengklaim bahwa BKMT dibentuk hanya untuk mendukung partai tertentu menjelang pemilu. Dalam unggahannya di Instagram, Anies menulis bahwa BKMT “bukan wadah pendidikan, melainkan alat politik”, sebuah pernyataan yang langsung menuai kontroversi. Beberapa tokoh politik disebut hadir hanya untuk kepentingan pencitraan semata."

In [18]:
label, berita_asli, skor = deteksi_beritas(hoax)

In [19]:
print("Hasil prediksi:", label)
print("Kemiripan dengan berita asli: {:.2f}%".format(skor))
print("Berita asli yang paling mirip:\n", berita_asli)

Hasil prediksi: Diduga Hoax
Kemiripan dengan berita asli: 63.43%
Berita asli yang paling mirip:
 anies di milad bkmt pengajian menghasilkan ibu-ibu berpengetahuan mantan gubernur dki jakarta anies baswedan menghadiri acara tasyakuran milad ke-42 tahun badan kontak majelis taklim bkmt di istora senayan jakarta selasa 21 2 dia pun memuji eksistensi ibu-ibu pengajian mewujudkan keberhasilan pendidikan di dalam keluarga ia mengatakan selama 42 tahun usianya bkmt telah menjadi teladan keberhasilan pendidikan dalam keluarga bkmt kata dia menjadi bukti bahwa pengajian menghasilkan ibu-ibu yang lebih berpengetahuan bkmt menjadi bukti bahwa pengajian menghasilkan ibu-ibu yang lebih berpengetahuan ibu-ibu yang punya bekal untuk mendidik anak-anaknya membuat rumah yang mencerminkan nilai islam dan akhlak yang baik kata anies dikutip dari unggahan akun media sosial instagram miliknya dalam unggahannya anies menyinggung soal bkmt yang tidak dapat dilepaskan dari sosok tuty alawiyah menurut anies tu